In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import time
from selenium.webdriver.support.ui import Select

# Academic Ranking of World Universities
# Shanghai Ranking

## Web scraping dos dados do ranking Shanghai

PAra o web scraing dos dados será usada a biblioteca selenium

In [5]:
website='https://www.shanghairanking.com/rankings/arwu/2023'
driver = webdriver.Chrome()
driver.get(website) #abre a pagina
driver.maximize_window()  #maximizar a página

Na primeira coluna da tabela estão os seguintes headers:

- World Rank;
- Institution;
- Country/Region;
- National/Regional Rank;
- Total Score;
- Menu com as feutures:
    - Alumni;
    - Award;
    - HiCi;
    - N&S;
    - PUB;
    - PCP.

As features estão em um menu dropdown. Portanto durante o processo de scraping deverá ocorrer interação com esse menu para que ser possível coletar os dados para cada uma delas. Além disso, os dados para todas as universidades estão 
em diversas páginas, sendo assim será necessário interagir com a numeração das páginas para coletar os dados do completos do ranking


In [105]:
features=["Alumni",'Award','HiCi','N&S','PUB','PCP'] #features calculadas cada universidade. O significado de cada uma dela pode ser encontrado no notebook com a análise dos dados dos rankings

#Dicionário que receberá os dados
ranking_2023={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}
 

In [106]:
#número de páginas
pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
paginas=pagination.find_elements('xpath','./li')
num_paginas=paginas[-2].text
num_paginas

'34'

In [109]:

botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]')
time.sleep(3)

pagina_atual=1
while pagina_atual<int(num_paginas):

    botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown. Ele está nos headers da tabela. Ao clicar nesse botão o menu dropdow é aberto
    botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
        
    opcoes=driver.find_elements(by='xpath',value='//th') #menus dropdown no header da tabela
    opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
    opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
    table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os header
    table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

    first_feature=True
    for feature in features:       
        botoes_header.click() #clicar no botão para abrir o menu dropdown
        time.sleep(1)
        for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature
                opcao.click()
                if first_feature:
                    for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking_2023['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                        ranking_2023['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                        codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]')
                        ranking_2023['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0])
                        ranking_2023['Regional Rank'].append(td[3].text)
                        ranking_2023['Total Score'].append(td[4].text)
                        ranking_2023[feature].append(td[5].text)
                        first_feature=False
                else:
                    for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking_2023[feature].append(td[5].text)
                
                time.sleep(5)
                break
    botao_next.click()
    botao_top=driver.find_element('xpath','//div[@class="back-top"]')
    botao_top.click()
    time.sleep(5)
    print(pagina_atual)
    pagina_atual+=1
              


1
2
3
4
5
6
7
8
9


ElementClickInterceptedException: Message: element click intercepted: Element <img data-v-24ba571f="" src="data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI0LjAuMSwgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IuWbvuWxgl8xIiB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB4PSIwcHgiIHk9IjBweCIKCSB2aWV3Qm94PSIwIDAgMTQgMTQiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE0IDE0OyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+CjxzdHlsZSB0eXBlPSJ0ZXh0L2NzcyI+Cgkuc3Qwe2ZpbGw6I0FFQUVBRTt9Cjwvc3R5bGU+CjxwYXRoIGNsYXNzPSJzdDAiIGQ9Ik0xMC45LDUuM0w3LjcsOS4yYy0wLjQsMC40LTEsMC40LTEuNCwwTDMuMSw1LjNDMi45LDUsMy4xLDQuNSwzLjUsNC41aDcuMUMxMC45LDQuNSwxMS4xLDUsMTAuOSw1LjN6Ii8+Cjwvc3ZnPgo=" alt="" class="nav-icon" style="right: 5px; top: 11px; width: 10px;"> is not clickable at point (823, 5). Other element would receive the click: <a data-v-0e4

In [90]:
botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown. Ele está nos headers da tabela. Ao clicar nesse botão o menu dropdow é aberto
botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
        
opcoes=driver.find_elements(by='xpath',value='//th') #menus dropdown no header da tabela
opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
    
table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os header
table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

first_feature=True
for feature in features:       
    botoes_header=driver.find_elements(by='xpath',value='//th//img') #clicar no botão para abrir o menu dropdown
    botoes_header=botoes_header[-1]
    botoes_header.click()
    time.sleep(2)
    for opcao in opcoes: #cada opcao é uma das features no menu dropdown
        if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature
            opcao.click()
            if first_feature:
                for row in table_rows: #loop por todas as linhas da tabela
                    td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                    ranking_2023['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                    ranking_2023['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                    codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]')
                    ranking_2023['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0])
                    ranking_2023['Regional Rank'].append(td[3].text)
                    ranking_2023['Total Score'].append(td[4].text)
                    ranking_2023[feature].append(td[5].text)
                    first_feature=False
            else:
                for row in table_rows: #loop por todas as linhas da tabela
                    td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                    ranking_2023[feature].append(td[5].text)
            
            time.sleep(5)
            break

In [41]:
pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")

In [46]:
botao_next=pagination.find_elements('xpath','./li[contains(@class,"next")]')

In [48]:
botao_next

[<selenium.webdriver.remote.webelement.WebElement (session="7d5bffbbbbb7fab50e7e2c26aac77d30", element="aa65c43a-d586-4183-8d46-d1e16b3c6b10")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d5bffbbbbb7fab50e7e2c26aac77d30", element="ad6b93e4-b109-4d1f-8692-dbb9de23eb87")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7d5bffbbbbb7fab50e7e2c26aac77d30", element="665bc794-77b4-46b2-be6a-93b2a46aa527")>]

In [58]:
botao_next[-1].click()

In [68]:
botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]')


In [70]:
botao_next.click()

In [4]:
for feature in features:       
    botoes_header=driver.find_elements(by='xpath',value='//th//img')
    botoes_header=botoes_header[-1]
    botoes_header.click()

    for opcao in opcoes:
        if opcao.text==feature:
            opcao.click()
            break
        time.sleep(5)
    

NameError: name 'driver' is not defined

In [104]:
botao_top=driver.find_element('xpath','//div[@class="back-top"]')
botao_top
botao_top.click()

In [10]:

table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os header
table_rows=box_tabela.find_elements(by='xpath',value="./tr") #rows da tabela

NameError: name 'box_tabela' is not defined

In [ ]:
#rows da tabela
table_rows=box_tabela.find_elements(by='xpath',value="./tr")

In [ ]:
td=table_rows[0].find_elements(by='xpath',value='./td')
td

[<selenium.webdriver.remote.webelement.WebElement (session="e20d329ccd9cc21e81f1773b8f8713c7", element="13f0c55b-9f41-4be0-a301-4d60ba4b1de5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e20d329ccd9cc21e81f1773b8f8713c7", element="ad921455-bb6d-4bc9-bd9a-3691ca3976f0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e20d329ccd9cc21e81f1773b8f8713c7", element="b4d56309-c553-43cc-9787-90b5fdd3def8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e20d329ccd9cc21e81f1773b8f8713c7", element="1b8095f1-d716-403f-8412-66234a6eb19b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e20d329ccd9cc21e81f1773b8f8713c7", element="40e41827-b963-4c1b-905a-e6210927bfbf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e20d329ccd9cc21e81f1773b8f8713c7", element="92021748-168f-4dba-9b25-676b67d237b3")>]

In [ ]:
print(td[0].text)
print(td[1].text)
x=td[2].find_element(by='xpath',value='div[@class="region-img"]')
print(x.get_attribute("style").split('/')[-1].split('.')[0])
print(td[4].text)
print(td[5].text)



1
Harvard University
us
100.0
100.0


In [40]:
botoes_header=driver.find_elements(by='xpath',value='//th//img')
botoes_header=botoes_header[-1]

In [56]:
features=["Alumni",'Award','HiCi','N&S','PUB','PCP']

In [50]:
opcoes=driver.find_elements(by='xpath',value='//th')
opcoes=opcoes[-1]
opcoes=opcoes.find_elements('xpath','.//li')
opcoes

[<selenium.webdriver.remote.webelement.WebElement (session="497da787d7ea0eec099d4486245de22d", element="7a97d66b-603a-4edc-98e0-ded286bcd02e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="497da787d7ea0eec099d4486245de22d", element="cadf30c7-9fa3-412d-be05-40ccd9bf4a5d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="497da787d7ea0eec099d4486245de22d", element="38c7e874-7a7e-46a8-a6ab-c878a0b1d6b2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="497da787d7ea0eec099d4486245de22d", element="fe6f7beb-a819-4b6f-ab14-5be7b6870d4a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="497da787d7ea0eec099d4486245de22d", element="84aca737-921a-48ec-b23f-67c37451758c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="497da787d7ea0eec099d4486245de22d", element="a2a52de9-867f-4c29-9540-0b0c7a055571")>]

In [52]:
opcoes[1].text

'Award'

In [58]:
features=["Alumni",'Award','HiCi','N&S','PUB','PCP']
opcoes=driver.find_elements(by='xpath',value='//th')
opcoes=opcoes[-1]
opcoes=opcoes.find_elements('xpath','.//li')
opcoes

for feature in features:       
    botoes_header=driver.find_elements(by='xpath',value='//th//img')
    botoes_header=botoes_header[-1]
    botoes_header.click()

    for opcao in opcoes:
        if opcao.text==feature:
            opcao.click()
            break
        time.sleep(5)
    